<div style="font-size:16pt" font-weight="bold">Dogs vs. Cats, Keras CNN with transfer learning (Inception V3)</div><br>
<br> The structure of this notebook:
<ol>
<li>Create a model by transfer learning
<li>Load the dataset from kaggle
<li>Split the dataset into training set and validation set
<li>See sample images
<li>Data Augmentation
<li>Training
<li>Prediction
</ol>
<br>
Reference:<br> Thie code of this notebook is inspired by <a href="https://www.coursera.org/specializations/tensorflow-in-practice"> TensorFlow in Practice Specialization</a>.<br> The theoretical background is explained in <a href="https://www.coursera.org/specializations/deep-learning"> Deep Learning Specialization</a>. The lecuture videos are also available on <a href="https://www.youtube.com/channel/UCcIXc5mJsHVYTZR1maL5l9w">youtube</a>. Both are offered by deeplearning.ai

In [ ]:
# Open this notebook on google colab or kaggle kernel
# Change runtime type to GPU
# In December 2019, tensorflow 2.0 is only available in google colab or kaggle kernel
# ! pip install tensorflow==2.0.0

In [ ]:
# Import the relevant libraries
import os
import json
import zipfile
import shutil
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from shutil import copyfile

print(tf.__version__)

<hr>

<div style="font-size:14pt" font-weight="bold">Step 1. Create a model by transfer learning</div><br>
Transfer learning is a technique to use parameters of a pre-trained model. In image classification, there are many models pre-trained on <a href="http://www.image-net.org/"> ImageNet</a>.  ImageNet is a database of images which has more than 14 million images with 20,000 categories. Several models including ResNet50 and InceptionV3 are available in <a href="https://keras.io/applications/"> keras</a>. <br>
For theortical background of CNN(convolutional neural network) and transfer learninng, deeplearning.ai lecture videos by professor Andrew Ng are very helpful.
Please find lecture #1-20 in the <a href="https://www.youtube.com/playlist?list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF"> link </a>. 

In [ ]:
# Import the inception V3 model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Clear any exiting model
tf.keras.backend.clear_session()

pre_trained_model = InceptionV3(input_shape = (150,150,3), # reshape images to 150 by 150 by 3 channels
                                include_top = False, # get straight to the CNN layer
                                weights = 'imagenet') # use the builed-in weight pre-trained on imagent

# Make all the layers in the pre-trained model non-trainable/frozen
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
# Print the InceptionV3 model summary
# pre_trained_model.summary()

In [ ]:
# Pick one layer from the miffde of the model
# We well build new dense layers after this layer

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Define a Callback class that stops training once accuracy reaches a threshold

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.875):
      print("\nReached 87.5% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Add layers after the picked last_layer
# Note that the layers below are trainable. 
# On the other hand, the layers up to the last_output is fronzen and non-trainable.

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for binary classification
x = layers.Dense(1, activation='sigmoid')(x)           

# instantiate a new model 
model = Model(pre_trained_model.input ,x) 

# Compile a model
model.compile(optimizer = RMSprop(lr=0.0001),  # We use RMSprop in this case
              loss = 'binary_crossentropy',  # cats vs. dogs is a binary problem
              metrics = ['acc']) # 'acc' stands for accuracy

# See the summary of the model 
# model.summary()

Droptout: please find lecture #6 in the <a href="https://www.youtube.com/watch?v=D8PJAL-MZv8&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=7&t=0s"> link </a>.<br>
RMSProp:  lecture #21 in the <a href="https://www.youtube.com/watch?v=_e-LFe_igno&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=22&t=0s"> link </a>.

So far, the model has learned nothing from the Dogs vs. Cats dataset. 
<hr>

<div style="font-size:14pt" font-weight="bold">Step2.1 Load the dataset from kaggle</div><br> 
If you are in the kaggle kernel implement the followings:

In [ ]:
print(os.listdir("../input/dogs-vs-cats"))

In [ ]:
# Delete the directory we will create if it already exists
try:
    shutil.rmtree("/tmp/cat_or_dog/")    
except:
    pass

# Create a new directory for the dataset
try:
    os.mkdir("/tmp/cat_or_dog/")
except:
    pass

In [ ]:
# Unzip　the training data
local_zip = '../input/dogs-vs-cats/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cat_or_dog')
zip_ref.close()

In [ ]:
os.listdir('/tmp/cat_or_dog')

<div style="font-size:14pt" font-weight="bold">If you are inside the kaggle kernel, Skip Step2.2 and jump tp Step 2.3</div><br>
> <hr>

<div style="font-size:14pt" font-weight="bold">Step2.2 Load the dataset from kaggle</div><br> 
If yur are o[](http://)utside the kaggle kernel, implement the following:

In [ ]:
# Login to kaggle 
# Join the competition where the data are stored
# Go to My account and click "Create new API Token" 
# A json file will be downloaded. It can be opened in any text file
# Set your kaggle username and key in the dictionary below 


# api_token = {"username":"your_kaggle_username","key":"your_kaggle_key"}

In [ ]:
# Create a .kaggle　directory in the root directory 


# try:
#     os.mkdir("/root/.kaggle/")
# except:
#     pass

In [ ]:
# Create a json file which stores the api keys


# with open('/root/.kaggle/kaggle.json', 'w') as file:
#     json.dump(api_token, file)

In [ ]:
# Set owner permission to the json file 

# ! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Delete the directory we will create if it already exists

# try:
#     shutil.rmtree("/tmp/cat_or_dog/")    
# except:
#     pass

# # Create a new directory for the dataset
# try:
#     os.mkdir("/tmp/cat_or_dog/")
# except:
#     pass

In [ ]:
# Go to the data tab in the kaggle competition and copy the command  
# After -p is the directory to store the data

# ! kaggle competitions download -c dogs-vs-cats -p /tmp/cat_or_dog/

In [ ]:
# Unzip　the training data


# local_zip = '/tmp/cat_or_dog/train.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/tmp/cat_or_dog')
# zip_ref.close()

<div style="font-size:14pt" font-weight="bold">Step2.3 Load the dataset from kaggle</div><br> 
The code below are the same for users outside the kaggle or inside the kaggle kernel

In [ ]:
# Cats and dogs images are stored in the same dirctory
print(os.listdir('/tmp/cat_or_dog'))
print(os.listdir('/tmp/cat_or_dog/train')[:10])
files_list = os.listdir('/tmp/cat_or_dog/train')

In [ ]:
# Create directories for training and validation

base_train_dir = '/tmp/cat_or_dog/training/'
base_val_dir = '/tmp/cat_or_dog/validation/'

# Delete the directory we will create if it already exists
try:
    shutil.rmtree(base_train_dir)    
except:
    pass

try:
    shutil.rmtree(base_val_dir)    
except:
    pass

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(base_train_dir, 'cats/')
train_dogs_dir = os.path.join(base_train_dir, 'dogs/')

# Directory with our validation cat/dog pictures
val_cats_dir = os.path.join(base_val_dir, 'cats/')
val_dogs_dir = os.path.join(base_val_dir, 'dogs/')


try:
    os.mkdir(base_train_dir)
    os.mkdir(base_val_dir)
    os.mkdir(train_cats_dir)
    os.mkdir(train_dogs_dir)
    os.mkdir(val_cats_dir)
    os.mkdir(val_dogs_dir)  
except:
    pass

In [ ]:
# When using .flow_from_directory in the generator,  the "training" directory should have sub-directories for the classes, "dogs" and "cats"
# Another option is to use .flow_from_dataframe. In this case, a dataframe with filename and corresponding categories is required. 
print(os.listdir(base_train_dir))

# Same for the "validation" directory 
print(os.listdir(base_val_dir))

In [ ]:
# Shuffle the files before splting into training set and test set
random.seed(0)
random_files_list = random.sample(files_list,len(files_list))
print(random_files_list[:10])
print(len(files_list))
print(len(random_files_list))

In [ ]:
# Deleat empty files
# In this dataset, there is no empty files
SOURCE = '/tmp/cat_or_dog/train/'

random_files_clean_list = []

for filename in random_files_list:
    file = os.path.join(SOURCE + filename)
    if os.path.getsize(file) > 0:
        random_files_clean_list.append(filename)
    else:
        print(filename + " is zero length, so ignoring.")

# print(random_files_clean_list)
print(len(random_files_list))
print(len(random_files_clean_list))

<hr>

<div style="font-size:14pt" font-weight="bold">Step 3. Split the dataset into training set and validation set</div><br> 
For details about training set and validation set, please find Andrew Ng's lecture #1 in the <a href="https://www.youtube.com/watch?v=1waHlpKiNyY&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=2&t=8s"> link</a>, or #5 in the <a href="https://www.youtube.com/watch?v=M3qpIzy4MQk&list=PLkDaE6sCZn6E7jZ9sN_xHwSHOdjUxUW_b&index=6&t=0s"> link</a>.

In [ ]:
# Split the files into training set and validation set

# Set the ratio for validation set
val_ratio = 0.3 

training_list = random_files_clean_list[:int(len(random_files_clean_list)*(1 - val_ratio))]
val_list = random_files_clean_list[-int(len(random_files_clean_list)*(val_ratio)):]

print(len(training_list))
print(len(val_list))
print(len(training_list) + len(val_list))

In [ ]:
# Create the training set

SOURCE = '/tmp/cat_or_dog/train/' # the current directory where files are stored

for filename in training_list:
    this_file = os.path.join(SOURCE + filename)
    cat_destination = os.path.join(train_cats_dir + filename)
    dog_destination = os.path.join(train_dogs_dir + filename)
    if 'cat' in filename:
        copyfile(this_file, cat_destination)
    elif 'dog' in filename:
        copyfile(this_file, dog_destination)

In [ ]:
train_dog_fnames = os.listdir(train_dogs_dir)
train_cat_fnames = os.listdir(train_cats_dir)


print(os.listdir(train_cats_dir)[:5])
print(os.listdir(train_dogs_dir)[:5])
print(len(os.listdir(train_cats_dir)))
print(len(os.listdir(train_dogs_dir)))

In [ ]:
# Create the validation set

SOURCE = '/tmp/cat_or_dog/train/' # the current directory where files are stored

for filename in val_list:
    this_file = os.path.join(SOURCE + filename)
    cat_destination = os.path.join(val_cats_dir + filename)
    dog_destination = os.path.join(val_dogs_dir + filename)

    
    if 'cat' in filename:
        copyfile(this_file, cat_destination)
    elif 'dog' in filename:
        copyfile(this_file, dog_destination)

In [ ]:
# print(os.listdir(val_cats_dir))
# print(os.listdir(val_dogs_dir))
print(len(os.listdir(val_cats_dir)))
print(len(os.listdir(val_dogs_dir)))

<hr>

<div style="font-size:14pt" font-weight="bold">Step 4. See sample images</div><br>

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
# The 8 images on the upper half are dogs, others are cats 
fig = plt.gcf()
fig.set_size_inches(ncols*3, nrows*3)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

<hr>

<div style="font-size:14pt" font-weight="bold">Step 5. Data Augmentation</div><br>
Data augmentation is a technique to artifically create training data. It rotates, shifts, zooms, shears, or horizontally flips the training images. In keras ImageDataGenerator class, the original images are unchaged. It just implements data augmentation when they are feeded into the training/fitting.
For details, please find Andrew Ng's lecture #21 in the <a href="https://www.youtube.com/watch?v=JI8saFjK84o&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF&index=22&t=0s"> link </a>.

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255., # rescale the images within 0-1. This yield better results when handling images. 
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented
test_datagen = ImageDataGenerator(rescale = 1.0 / 255.0)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(base_train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(base_val_dir,
                                                         batch_size = 20,
                                                         class_mode = 'binary',
                                                         target_size = (150,150))

In [ ]:
# See the indices of the classes
# In keras ImageDataGenerator class, classes are indexed in alphabetical order
train_generator.class_indices

<hr>

<div style="font-size:14pt" font-weight="bold">Step 6. Training</div><br>
When implementing training, it is very important to understand the concept of bias/underfitting and variance/overfitting. When a model has high bias, it has low performance both on the training set and the validation set. If the model has high variance, the model performs very well on the training set but has low performance on the validation set. <br><br>When the model is underfitting, there are two solutions among others. One way is to do more training or train on more epochs. If it still not work well, we have to change the model architecture. For example, if the target objects are not in the center of the images, neural networks without convolutional layers tend to have poor performance (high bias). In that case, CNN is a better choice.  <br><br>
When the model is overfitting, there are muliple choices. Regularization/add penalties, drop out, or training on fewer epochs are among them. Data augmentation is also a way to reduce overfitting. <br><br>
Bias and Variance: Please find Andrew Ng's lecture #2 in the <a href="https://www.youtube.com/watch?v=SjQyLhQIXSM&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc&index=2">link</a>.<br>


In [ ]:
# Fit the model the training data
# Take the log of the training in history
callbacks = myCallback()
history = model.fit_generator(
    generator = train_generator, # feed the training data via the generator
    steps_per_epoch = 8, # this is the batch size. parameters are updated per this batch size
    epochs = 100, # number of cycles. In one epoch, the whole dataset is used once.
    verbose = 2, # print out the logs
    callbacks = [callbacks], # use callbacks we set before
    validation_data = validation_generator, # feed the validation data via the generator
    validation_steps = 50 
)

In [ ]:
# Plot the accuracy history 
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

<hr>

<div style="font-size:14pt" font-weight="bold">Step 7. Prediction</div><br>
Prediction of the unlabeled test set images is the final stage. 

In [ ]:
# Inside the kaggle kernel
# Unzip　the test data
local_zip = '../input/dogs-vs-cats/test1.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cat_or_dog')
zip_ref.close()

In [ ]:
# Outside the kaggle kernel
# Unzip　the test data
# local_zip = '/tmp/cat_or_dog/test1.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/tmp/cat_or_dog')
# zip_ref.close()

In [ ]:
base_test_dir = '/tmp/cat_or_dog/test1/'

# print(os.listdir(base_test_dir))

In [ ]:
# To use .flow_from_dataframe. in the generator, we have to create a dataframe  
test_filenames = os.listdir(base_test_dir)
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
# Create a generator for test set
test_gen = ImageDataGenerator(rescale=1./255.0)
batch_size=20

test_generator = test_gen.flow_from_dataframe(
    test_df, 
    base_test_dir, 
    x_col='filename',
    y_col=None,
    class_mode=None,
    batch_size=batch_size,
    target_size=(150, 150),
    shuffle=False
)

In [ ]:
# Predict the test set
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
threshold = 0.5
test_df['category'] = np.where(predict > threshold, 1,0) # we can also 

In [ ]:
# See sample results, # 1 = dog, 0 = cat
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(9, 18))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img(base_test_dir+filename, target_size=(150,150))
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' ) 
plt.tight_layout()
plt.show()

<div style="font-size:14pt" font-weight="bold">Submit (from outside the kaggle)</div><br>
<br>
Note that we cannot submit past competiton.

In [0]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('/tmp/cat_or_dog/submission_20191230.csv', index=False)

In [0]:
!  kaggle competitions submit -c dogs-vs-cats -f /tmp/cat_or_dog/ submission_20191229.csv -m "First submission"